# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

     |████████████████████████████████| 320.4 MB 40 kB/s 
     |████████████████████████████████| 2.9 MB 45.6 MB/s 
     |████████████████████████████████| 20.1 MB 11.2 MB/s 
     |████████████████████████████████| 459 kB 43.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:


     |████████████████████████████████| 52 kB 770 kB/s 


# 1. Test Random Environment with OpenAI Gym

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import sympy

In [3]:
a = np.array([0,1,2,3,4,5,6,7,8,9])
a[(6 - 6%4):(6 - 6%4 + 4)] = [0,0,0,0]
a

array([0, 1, 2, 3, 0, 0, 0, 0, 8, 9])

In [4]:
sample_state = [
    [1, 2, -3], # 1x + 2y - 3 = 0   <- first equation
    [3, 0, -6]  # 3x - 6 = 0        <- second equation
]
def state_to_equation(eqNr):
    state = sample_state[eqNr]
    left = "{} * x + {} * y + {}".format(*state)
    return left, 0
    
left1, right1 = state_to_equation(0)
eq1 = sympy.Eq(sympy.sympify(left1), right1)
print(eq1)
               
left2, right2 = state_to_equation(1)
eq2 = sympy.Eq(sympy.sympify(left2), right2)

x,y = sympy.symbols('x y')
result = sympy.linsolve([eq1, eq2], (x, y))
result

Eq(x + 2*y - 3, 0)


FiniteSet((2, 1/2))

In [10]:
import math
import random
from functools import reduce
from sympy.ntheory import factorint
from scipy.stats import norm
import re
lista = [15, 30, 10]

In [11]:
from time import perf_counter as pc

In [12]:
# Discrete?

In [287]:
class MathEquationEnv(Env):
    
    def __init__(self, equation_type: str, number_of_equations: int):
        self.number_of_equations = number_of_equations
        self.observation_space_number, self.equation = self.state_evaluation(equation_type)
        self.observation_space_equation = self.observation_space_number * 1
        self.observation_space_number = self.number_of_equations * self.observation_space_number

        ## action space is equal to 2 * number of equations * number of variables in the equation
        self.action_space = Discrete(2*self.observation_space_number)
        ## observation_space is equal to number of equations * number of variables in the equation
        self.observation_space = Discrete(self.observation_space_number)
        self.state = self.random_state()
        
        ###R
        self.previous_state_value = 0
        self.pretty_denominators = [1,2,3,4,5,10,20,25,100]
        self.previous_step_denominators = [0,0]
        self.previous_step_solvable = 0
        ###

        # print("Initial state:")
        # print(self.state_to_equation(0)[0])
        # print(self.state_to_equation(1)[0])
        # print()

        
        self.generator_length = 60
        self.variables = sympy.symbols(self.get_symbols(equation_type))
        print(self.variables)
        x,y = sympy.symbols('x y')
        self.x = x
        self.y = y
    
    def get_symbols(self, equation):
        return " ".join(set(re.findall(r"\b[a-z]{1}\b", equation)))

    def state_evaluation(self, equation_type: str) -> (int, str):
        pattern = re.compile(r"[^ +]+")
        strings = pattern.findall(equation_type)
        strings.sort()
        strings.sort(key=len)
        copy_s = ["" for x in range(len(strings))]
        operations = 0
        for i, string in enumerate(strings):
            if any(x in string for x in ("sin", "cos", "tg")):
                copy_s[i] = string.replace("(", "(1/2 * pi * {}*")
                copy_s[i] = "{} * " + copy_s[i]
                operations += 2
                continue
            copy_s[i] = "{} * " + strings[i]
                
            operations+=1
        

        print(" + ".join(copy_s))
        return (operations, " + ".join(copy_s))

    def get_all_equations(self):
        equations = []
        for i in range(self.number_of_equations):
          left, right = self.state_to_equation(i)
          eq = sympy.Eq(sympy.sympify(left), right)
          equations.append(eq)
        return equations
    
    def random_state(self):
        random_state = random.choices(range(-10, 10), k=self.observation_space_number)
        return random_state
    
    def state_to_equation(self, eqNr):
        eq_state = self.state[self.observation_space_equation * eqNr:self.observation_space_equation * (eqNr+1)]
        left = self.equation.format(*eq_state)
        #print(left)
        return left, 0

            
    def step(self, action):
        """A step
        """

        
        self.generator_length-=1
        info = {}
        reward = 0
        # if action in (0,2,5,7,10,12,15,17):
        #   action = random.choice((1,3,4,6,8,9))
        self.state[action%self.observation_space_number] += 1 if action//self.observation_space_number else -1
            
        ##R
        equations = self.get_all_equations()
        ##R

        # #x,y = sympy.symbols('x y')
        # left1, right1 = self.state_to_equation(0)
        # eq1 = sympy.Eq(sympy.sympify(left1), right1)
                
        # left2, right2 = self.state_to_equation(1)
        # eq2 = sympy.Eq(sympy.sympify(left2), right2)
        
        done = False      

        ##ending was here
        
        #print(sympy.sympify(left1), right1)
        #print(sympy.sympify(left2), right2)
        #print('RESULT', eq1, eq2)
        if any(isinstance(equation, bool) for equation in equations):
            return self.state, reward, done, info
        
        # if eq1 == True or eq2 == True or eq1 == False or eq2 == False:
        #     return self.state, reward, done, info
        
        t0 = pc()
        try:
            t0 = pc()
            # result = sympy.nsolve([eq1, eq2], (self.x, self.y), (1,1))
            result = sympy.solve(equations, self.variables)
            # print(result, type(result))
            # print(pc() - t0,"\n", eq1, "\n", eq2, type(eq1))
        except Exception:
            result = None

        ###R

        coeffs = [self.state[self.observation_space_equation * x:self.observation_space_equation * (x+1)] for x in range(self.number_of_equations)]
        # coeffs1 = self.state[:5]
        # coeffs2 = self.state[5:]
        
        gcds = [reduce(lambda a,b: math.gcd(a,b), coeffs_reduce) for coeffs_reduce in coeffs]
        # gcd_equation1 = reduce(lambda a,b,: math.gcd(a,b), coeffs1)
        # gcd_equation2 = reduce(lambda a,b,: math.gcd(a,b), coeffs2)
        
        number_of_divisors = [sum(factorint(gcd).values()) for gcd in gcds]
        # numbers_of_divisors1 = sum(factorint(gcd_equation1).values())
        # numbers_of_divisors2 = sum(factorint(gcd_equation2).values())
        
        value1 = sum( [ norm(6, 2).cdf(divisor_number) for divisor_number in number_of_divisors] ) * 10
        
        reward += ( value1 - self.previous_state_value)

        self.previous_state_value = value1


        #### NEED expansion and more sofisticated handling of non 1 value results

        # try:
        #   if result:
        #     if result[self.x].q in self.pretty_denominators:
        #       if self.previous_step_denominators[0] != 1:
        #         reward += 1
        #         self.previous_step_denominators[0] = 1
        #     else:
        #       if self.previous_step_denominators[0] != -1:
        #         reward -= 1
        #         self.previous_step_denominators[0] = -1

        #     if result[self.y].q in self.pretty_denominators:
        #       if self.previous_step_denominators[1] != 1: 
        #         reward += 1
        #         self.previous_step_denominators[1] = 1
        #     else:
        #       if self.previous_step_denominators[1] != -1:
        #         reward -= 1
        #         self.previous_step_denominators[1] = -1
        # except:
        #   print("Yikes", result)
        
        ####
         

        
        # print(value, value - self.previous_state_value)
        
        # value =+ norm(6, 2).cdf(numbers_of_divisors1)
        # value =+ norm(6, 2).cdf(numbers_of_divisors2)
        
        ## - Harder than I thought (Will have to do some validation beforehand)
        # if isinstance(tuple, result):
        #   pretty_results = sum( map(lambda x: 1 if x.q in self.pretty_denominators else 0, result) )
        # elif isinstance(dict, result):
        #   ##implement(easier than the first one)
        #   pass

        ###

        if self.generator_length <= 0:
            print("Final state:")
            for i in range(self.number_of_equations):
              print(self.state_to_equation(i)[0])
            print(result)
            done = True

        #print(result)
        if not result:
            if self.previous_step_solvable != -1:
              reward -= 1
              self.previous_step_solvable = -1
        else:
            if self.previous_step_solvable != 1:
              reward += 1
              self.previous_step_solvable = 1
            # print("Final state:")
            # print(self.state_to_equation(0)[0])
            # print(self.state_to_equation(1)[0])
            # print()
            # done = True
        
        ###R
        # self.previous_step_state[ action % 10 ] += 1 if action // 10 else -1
        ###
            
        return self.state, reward, done, info
        
    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.state = self.random_state()
        print("Initial state:")
        for i in range(self.number_of_equations):
          print(self.state_to_equation(i)[0])
        print()
        self.generator_length = 60
        return self.state

In [358]:
## Do not make additional spaces in the () brackets just the variables like:
env = MathEquationEnv("sin(x) + cos(x) + 1", 1)
env = MathEquationEnv("x + y + z", 3)

{} * 1 + {} * cos(1/2 * pi * {}*x) + {} * sin(1/2 * pi * {}*x)
x
{} * x + {} * y + {} * z
(x, z, y)


In [359]:
env.observation_space.sample()

4

In [360]:
x,y = 0.522022196614492, -0.915915828629164
print(8 * x**2 + 5*x -6*y**2 -9*y -8)

1.7763568394002505e-15


In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        #print('hmm')
        action = env.action_space.sample()
        
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Initial state:
1 * x + -1 * y + -9 * z
1 * x + 4 * y + -8 * z
-7 * x + -1 * y + -7 * z

Final state:
-1 * x + 0 * y + -11 * z
0 * x + 4 * y + -7 * z
-5 * x + 0 * y + -7 * z
{x: 0, z: 0, y: 0}
Episode:1 Score:1.0404969409489029
Initial state:
-5 * x + -7 * y + -4 * z
2 * x + -3 * y + -2 * z
0 * x + -10 * y + 2 * z

Final state:
-7 * x + -7 * y + -7 * z
2 * x + 0 * y + -2 * z
0 * x + -13 * y + 3 * z
{x: 0, z: 0, y: 0}
Episode:2 Score:0.09719534588292075
Initial state:
-4 * x + 2 * y + -7 * z
2 * x + 4 * y + 5 * z
5 * x + 1 * y + -8 * z

Final state:
-7 * x + 0 * y + -8 * z
5 * x + 6 * y + 9 * z
4 * x + 5 * y + -6 * z
{x: 0, z: 0, y: 0}
Episode:3 Score:-0.09719534588292078
Initial state:
-2 * x + 5 * y + 6 * z
2 * x + 1 * y + 2 * z
2 * x + 1 * y + -5 * z

Final state:
4 * x + 3 * y + 5 * z
-2 * x + 3 * y + 0 * z
6 * x + 4 * y + -5 * z
{x: 0, z: 0, y: 0}
Episode:4 Score:0.0
Initial state:
-8 * x + 3 * y + -5 * z
8 * x + -7 * y + -10 * z
-4 * x + -7 * y + 1 * z

Final state:
-11 * x + 0 * y

# 2. Create a Deep Learning Model with Keras

In [331]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [332]:
env.observation_space.n

20

In [333]:
states = env.observation_space.shape
print(states)
actions = env.action_space.n

()


In [334]:
env.action_space

Discrete(40)

In [335]:
def build_model(observations, actions):
    model = Sequential()    
    model.add(Dense(observations, activation='relu', input_shape=(1, observations) )  )
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    model.add(Flatten())
    return model

In [141]:
del model 

In [336]:
def quick_building(env):
    observation_space = env.observation_space.n
    action_space = env.action_space.n
    return build_model(observation_space, action_space)

In [338]:
## Here you create a model with whatever sample equation you want
## the variables can only consist of 1 letter so variable x1 or y2 is not permited
## + 1 at the end is also important, include it in the equation
## if you want a free variable to change
env = MathEquationEnv("x + y + 1", 2)
model = quick_building(env)

{} * 1 + {} * x + {} * y
(x, y)


In [339]:
model.input_shape

(None, 1, 6)

In [340]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 1, 6)              42        
_________________________________________________________________
dense_53 (Dense)             (None, 1, 24)             168       
_________________________________________________________________
dense_54 (Dense)             (None, 1, 12)             300       
_________________________________________________________________
flatten_17 (Flatten)         (None, 12)                0         
Total params: 510
Trainable params: 510
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [341]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [342]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [343]:
def quick_building_agent(model, env):
    actions = env.action_space.n
    return build_agent(model, actions)

In [344]:
tuple(model.input.shape)

(None, 1, 6)

In [345]:
dqn = quick_building_agent(model, env)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)

Training for 5000 steps ...
Initial state:
0 * 1 + 2 * x + 4 * y
6 * 1 + -7 * x + 8 * y

Interval 1 (0 steps performed)
   11/10000 [..............................] - ETA: 23:11 - reward: 0.1022

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


   59/10000 [..............................] - ETA: 24:40 - reward: 0.0182Final state:
-4 * 1 + 2 * x + 10 * y
-11 * 1 + -22 * x + 8 * y
{x: -39/118, y: 55/118}
   60/10000 [..............................] - ETA: 24:25 - reward: 0.0179Initial state:
-9 * 1 + -8 * x + 6 * y
-5 * 1 + -9 * x + -4 * y

  119/10000 [..............................] - ETA: 16:38 - reward: 0.0086Final state:
-10 * 1 + -9 * x + 8 * y
-31 * 1 + -30 * x + -3 * y
{x: -278/267, y: 7/89}
  120/10000 [..............................] - ETA: 16:35 - reward: 0.0086Initial state:
4 * 1 + -6 * x + -1 * y
-1 * 1 + -7 * x + -9 * y

  179/10000 [..............................] - ETA: 13:47 - reward: 0.0060Final state:
6 * 1 + -2 * x + -6 * y
-2 * 1 + -5 * x + -3 * y
{x: -5/4, y: 17/12}
  180/10000 [..............................] - ETA: 13:46 - reward: 0.0060Initial state:
-10 * 1 + -5 * x + 7 * y
8 * 1 + 5 * x + -7 * y

  239/10000 [..............................] - ETA: 12:29 - reward: 0.0045Final state:
-5 * 1 + -1 * x + 

In [ ]:
fscores = dqn.test(env, nb_episodes=50, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 50 episodes ...
Initial state:
-8 * x + 7 * y + -9 * 1
1 * x + -10 * y + 6 * 1

Final state:
-8 * x + 7 * y + -69 * 1
1 * x + -10 * y + 6 * 1
{x: -648/73, y: -21/73}
Episode 1: reward: 0.000, steps: 60
Initial state:
1 * x + 9 * y + -3 * 1
3 * x + -6 * y + 7 * 1

Final state:
1 * x + 9 * y + -63 * 1
3 * x + -6 * y + 7 * 1
{x: 105/11, y: 196/33}
Episode 2: reward: 0.000, steps: 60
Initial state:
8 * x + -4 * y + -1 * 1
0 * x + -7 * y + 1 * 1

Final state:
8 * x + -4 * y + -61 * 1
0 * x + -7 * y + 1 * 1
{x: 431/56, y: 1/7}
Episode 3: reward: 0.000, steps: 60
Initial state:
7 * x + -10 * y + 6 * 1
9 * x + -7 * y + -6 * 1

Final state:
-26 * x + -37 * y + 6 * 1
9 * x + -7 * y + -6 * 1
{x: 264/515, y: -102/515}
Episode 4: reward: 0.000, steps: 60
Initial state:
4 * x + -2 * y + 6 * 1
-3 * x + -1 * y + 2 * 1

Final state:
4 * x + -36 * y + 6 * 1
-3 * x + -1 * y + 28 * 1
{x: 501/56, y: 65/56}
Episode 5: reward: 0.049, steps: 60
Initial state:
-5 * x + 4 * y + -7 * 1
5 * x + -2 * y

# 4. Reloading Agent from Memory

In [ ]:
import gym

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

NameError: ignored

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

NameError: ignored

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
